In [2]:
# local python tests
from pprint import pprint
from igraph import *
import igraph.test
import pickle
import os
import numpy as np
from scipy.sparse import csr_matrix
import collections
import copy
from random import randint





"""

    %%%%%%%%%%%%%%%%%%%%%%%%%%% gcn tests %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    x  feature vectors
    tx feature vectors for test (unlabeled)
    allx feature vectors for all

    y  label vectors       numpy.ndarray
    ty test label vectors  numpy.ndarray
    ally  test label of all

    graph : adjacency matrix (dict of lists)
    test.index: node id list for tests?


    steps
    1) inspect ind.citeseer.x/tx/y/ty/allx/ally/test.index

    2) build those files from code

    3) call learning algorithm
        cd src/notebooks
        python gcn_tests_input.py&&  cp ind.* ../gcn/gcn/gcn/data
        cd src/gcn/gcn/gcn
        python train.py --dataset t1


    4) look at embedding result

"""


def pickleToDisk(myobj, filepath):
    pickle.dump(myobj, open(filepath, "wb"))


def printGraphObject(filepath):


    myobject = os.path.basename(filepath)
    print("\n\n*************** "+myobject+" ********************")
    y = pickle.load(open(filepath,'rb'))
    print(type(y))
    if type(y) ==  type([]) or type(y) == type(np.ndarray([])):
        
        print(len(y))
        pprint(y[:min(3,len(y)-1)])
    elif type(y) == type({}) or type(y)==type(collections.defaultdict()):
        pprint(y.keys()[:min(3,len(y)-1)])
        pprint(y[y.keys()[0]])
        pprint(y[y.keys()[1]])
        pprint(y[y.keys()[20]])
        pprint(y[y.keys()[3000]])

    else:
        
        pprint(y.shape)
        pprint(y[0,:min(3,y.shape[1]-1)].toarray())
        pprint(y[0,0].shape)
        pprint(type(y[0,0]))
        pprint(y[0,0])

def readListFile(filepath):
    myobject = os.path.basename(filepath)
    print("\n\n*************** "+myobject+" ********************")
    

    testInstances =[]
    with open(filepath, 'rb') as f:
        for line in f.readlines():
            testInstances.append(int(line))

    pprint(testInstances[:min(3,len(testInstances)-1)])


def inspectData():

    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.y')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.ty')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.x')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.tx')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.graph')
    readListFile('../gcn/gcn/gcn/data/ind.citeseer.test.index')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.allx')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.ally')





def generateData(suffix, n_node=1000, n_test=200, n_train=300 ):
    
    

    #igraph.test.run_tests()

    # generation
    #g = Graph.Tree(127, 2)
    #g = Graph.GRG(100, 0.2)
    #g = Graph.Erdos_Renyi(100, 0.2)
    #g = Graph.Watts_Strogatz(1, 100, 4, 0.5 )
    g = Graph.Barabasi(n_node)
    summary(g)

    # graph metrics
    # pprint(g.degree([2,3,6,99]))
    # pprint(g.edge_betweenness())
    # pprint(g.pagerank())
    #pprint(g.get_adjacency())
    #pprint(dir(g))
    


    # test.index
    # list first 5 nodes -> to test.index
    testList = [] 
    with open(suffix+'.test.index','wb') as f:
        while len(testList) < n_test:
            j = randint(0, n_node-1)
            #verify it is not alerady there
            try:
                pos = testList.index(j)
            except:
                testList.append(j)
            f.write(str.encode(str(j)+"\n"))
            
    
    print(" len(testList)=",len(testList))
    #print(testList)

    # adjacency dict (graph)
    i = 0
    A = g.get_adjlist()
    graphAdj = {}
    for node in A:
        graphAdj[i] = node
        i+=1
    pickleToDisk(graphAdj, suffix+".graph")

    # features (allx, x, tx)
    #  no features -> a 0 for each node 
    #   or node degree
    #  into a sparce matrix
    xFeatures = g.degree()
    #print(xFeatures)
    #xFeatures = [feature * 1.0 for feature in xFeatures]
    xFeatures = np.array(xFeatures, np.float64)
    xFeatures = np.ones(n_node)
    
    # see if any degree is negative
    degs = np.array(xFeatures)
    #degs2 = degs < 0
    #degsum = degs2.sum(axis=0)
    #print(degsum)
    
    
    # allx is for labeled and unlabeled training samples
    allx  = copy.deepcopy([xFeatures[i] for i in range(n_node) if i not in testList])
    # x is for labeled training samples only
    x = copy.deepcopy(allx[:-500])
    #tx = copy.deepcopy(xFeatures[range(n_node-n_test, n_node)])
    tx = [xFeatures[i] for i in testList]
    
    print("verifying the len of x")
    print(len(range(n_node)))
    print(len([i for i in range(n_node) if i not in testList]))

    print(" len(x)=",len(x))
    """
    # remove test List
    # remove also validation
    #for elem in tx:
    #    x.pop(x.index(elem))
    for elindex in testList:
        x.pop(elindex)
    
    validationList = range(testList[-1]+1, testList[-1]+501)
    for elindex in validationList:
        x.pop(elindex)
    """
    
    allx = csr_matrix(allx)
    allx = np.transpose(allx)
    pickleToDisk(allx, suffix+".allx")
    x = csr_matrix(x)
    x = np.transpose(x)
    pickleToDisk(x, suffix+".x")
    tx = csr_matrix(tx)
    tx = np.transpose(tx)
    pickleToDisk(tx, suffix+".tx")

    print(x.shape)
    print(tx.shape)
    print(allx.shape)
    


    # labels (ally,y , ty)
    prs = g.pagerank()
    print(prs[:10])
    prs = np.array(prs)
    threshold = 0.07
    prs = prs >= threshold
    prs2 = prs < threshold

    prs = np.vstack((prs,prs2)).T
    pprint(prs)

    # labels of x, and empty labels of unlabeled training samples
    prs_all = copy.deepcopy([prs[i] for i in range(n_node) if i not in testList])
    # labels of the labeled training samples
    prs_train = copy.deepcopy(prs_all[:-500])
    prs_test = copy.deepcopy([prs[i] for i in testList])
    #testList.extend(validationList)
    #finalList = testList
    #prs_train = [ prs[i] for i in range(prs.shape[0]) if i not in finalList ]

    ally = np.array(prs_all)
    y = np.array(prs_train)  
    ty = np.array(prs_test)


    pickleToDisk(ally, suffix+".ally")
    pickleToDisk(y, suffix+".y")
    pickleToDisk(ty, suffix+".ty")
    
    print(y.shape)
    print(ty.shape)
    print(ally.shape)
    
    

    # run training on those files

In [3]:
generateData(suffix="ind.t3",n_node=4000, n_train=700, n_test=300)
!mv ind.* ./data/

IGRAPH U--- 4000 3999 -- 
 len(testList)= 300
verifying the len of x
4000
3700
 len(x)= 3200
(3200, 1)
(300, 1)
(3700, 1)
[0.01334611943003789, 0.001859572831569819, 0.007171704401611703, 0.0005931575756603369, 0.0049778513696610354, 0.0036925999983499487, 0.01035923017621399, 0.0009020922552866687, 0.00012409695813389278, 0.0023102452839413]
array([[False,  True],
       [False,  True],
       [False,  True],
       ...,
       [False,  True],
       [False,  True],
       [False,  True]])
(3200, 2)
(300, 2)
(3700, 2)


In [4]:
!~/.pyenv/shims/python  train.py --dataset t3


len(y): 3200
labels shape:
(4000, 2)
len(idx_train) 3200
len(idx_val) 500
len(idx_test) 316
features after row normalize and tuple
(array([[   0,    0],
       [   1,    0],
       [   2,    0],
       ...,
       [3997,    0],
       [3998,    0],
       [3999,    0]], dtype=int32),
 array([1., 1., 1., ..., 1., 1., 1.]),
 (4000, 1))
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

2019-01-21 21:04:58.665518: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(4000, 2)
array([[-0.5444204 ,  0.47993788],
       [-0.27381238,  0.13165545],
       [-0.26270205,  0.06640942],
       ...,
       [-0.08315901,  0.25856102],
       [ 0.02593395,  0.09328651],
       [-0.05842893,  0.11566696]], dtype=float32)
Epoch: 0001 train_loss= 0.62567 train_acc= 0.70469

Epoch: 0026 train_loss= 0.18485 train_acc= 0.97156 val_loss= 0.13076 val_acc= 1.00000 time= 0.00786
(4000, 2)
array([[-5.9659524 ,  6.4514723 ],
       [-2.2326097 ,  1.9711128 ],
       [-5.339393  ,  4.53113   ],
       ...,
       [-1.3303611 ,  1.3605826 ],
       [-0.8425091 ,  1.2685239 ],
       [-0.09243106,  0.17920375]], dtype=float32)
Epoch: 0027 train_loss= 0.17205 train_acc= 0.97781 val_loss= 0.12151 val_acc= 1.00000 time= 0.00806
(4000, 2)
array([[-5.6869826 ,  6.620798  ],
       [-2.1142755 ,  1.7786757 ],
       [-5.3059597 ,  5.4756665 ],
       ...,
       [-0.38418967,  0.25449702],
       [-0.50099975,  0.5273978 ],
       [-0.2134351 ,  0.39200896]], dtype=float32)
Epoch: 0028 train_loss= 0.16318 train_acc= 0.97969 val_loss= 0.11283 val_acc= 1.00000 time= 0.00971
(4000, 2)
array([[-5.9728427 ,  7.0228634 ],
       [-2.2402272 ,  2.8287537 ],
       [-4.5295625 ,  3.9889348 ],
       ...,
       [-0.38922673,  0.02204786],
       [-1.1843556 ,  0.8949149 ],
       

(4000, 2)
array([[-12.639979 ,  12.116109 ],
       [ -6.3338184,   6.396559 ],
       [ -9.087965 ,   9.219495 ],
       ...,
       [ -2.55593  ,   2.258484 ],
       [ -3.3648949,   2.6996868],
       [ -0.2517766,   0.3436592]], dtype=float32)
Epoch: 0051 train_loss= 0.07404 train_acc= 0.97562 val_loss= 0.01946 val_acc= 1.00000 time= 0.00898
(4000, 2)
array([[-13.420328 ,  13.85299  ],
       [ -6.426248 ,   6.276064 ],
       [ -8.509573 ,   8.57058  ],
       ...,
       [ -1.5986624,   2.7316284],
       [ -3.631215 ,   2.5626597],
       [ -3.43431  ,   5.3618464]], dtype=float32)
Epoch: 0052 train_loss= 0.08256 train_acc= 0.97656 val_loss= 0.01811 val_acc= 1.00000 time= 0.00803
(4000, 2)
array([[-13.169338 ,  12.751463 ],
       [ -6.99043  ,   7.245598 ],
       [ -9.352121 ,   9.805555 ],
       ...,
       [ -1.6823561,   2.1330488],
       [ -1.2794569,   1.3282468],
       [ -3.0220973,   2.1050253]], dtype=float32)
Epoch: 0053 train_loss= 0.07458 train_acc= 0.97938 val_l

(4000, 2)
array([[-17.814207  ,  18.07834   ],
       [ -6.877085  ,   6.5555034 ],
       [-13.539238  ,  13.615887  ],
       ...,
       [ -0.02058568,  -0.01881501],
       [ -0.34886444,   0.3037689 ],
       [ -1.8629092 ,   2.405338  ]], dtype=float32)
Epoch: 0076 train_loss= 0.05766 train_acc= 0.97312 val_loss= 0.00444 val_acc= 1.00000 time= 0.00945
(4000, 2)
array([[-17.843212 ,  18.801964 ],
       [ -8.463348 ,   7.457558 ],
       [-11.306553 ,  11.723909 ],
       ...,
       [ -2.4654765,   3.8047152],
       [ -3.2386508,   2.782722 ],
       [  0.       ,   0.       ]], dtype=float32)
Epoch: 0077 train_loss= 0.05429 train_acc= 0.97438 val_loss= 0.00425 val_acc= 1.00000 time= 0.00826
(4000, 2)
array([[-17.672852 ,  17.512772 ],
       [ -7.610646 ,   7.970842 ],
       [-14.069526 ,  13.488709 ],
       ...,
       [ -2.113163 ,   1.5015296],
       [ -6.023964 ,   5.023423 ],
       [ -0.8967995,   0.9240658]], dtype=float32)
Epoch: 0078 train_loss= 0.05706 train_acc= 0

Epoch: 0099 train_loss= 0.03924 train_acc= 0.97656 val_loss= 0.00203 val_acc= 1.00000 time= 0.00844
(4000, 2)
array([[-26.038673 ,  28.532629 ],
       [ -5.7577486,   6.2926865],
       [-19.206442 ,  21.477316 ],
       ...,
       [ -3.272184 ,   3.8057528],
       [ -1.9700072,   4.5281496],
       [ -7.7623224,   7.0039635]], dtype=float32)
Epoch: 0100 train_loss= 0.04975 train_acc= 0.97125 val_loss= 0.00199 val_acc= 1.00000 time= 0.01029
(4000, 2)
array([[-26.985413  ,  29.69516   ],
       [-11.49552   ,  13.852167  ],
       [-19.372353  ,  19.424015  ],
       ...,
       [ -1.2824867 ,   2.1467447 ],
       [ -3.5594635 ,   4.977041  ],
       [ -0.67965776,   0.96106076]], dtype=float32)
Epoch: 0101 train_loss= 0.03783 train_acc= 0.97781 val_loss= 0.00196 val_acc= 1.00000 time= 0.00816
(4000, 2)
array([[-21.430841 ,  25.011314 ],
       [-10.6748705,  12.672284 ],
       [-16.071762 ,  17.690758 ],
       ...,
       [ -4.235486 ,   5.064821 ],
       [ -4.038575 ,   4.32526

In [62]:
!~/.pyenv/shims/python  train.py --dataset cora


len(y): 140
labels shape:
(2708, 7)
len(idx_train) 140
len(idx_val) 500
len(idx_test) 1000
features after row normalize and tuple
(array([[   0, 1274],
       [   0, 1247],
       [   0, 1194],
       ...,
       [2707,  329],
       [2707,  186],
       [2707,   19]], dtype=int32),
 array([0.11111111, 0.11111111, 0.11111111, ..., 0.07692308, 0.07692308,
       0.07692308], dtype=float32),
 (2708, 1433))
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

2019-01-20 20:31:34.238614: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(2708, 7)
array([[ 5.7289202e-04,  6.7571541e-03,  5.7875710e-03, ...,
         5.6748353e-03,  1.1701141e-03, -8.5713860e-04],
       [-3.2951573e-03,  3.5759811e-03,  2.3557898e-04, ...,
        -6.5138435e-04, -1.4694517e-0

(2708, 7)
array([[-0.05860034,  0.03541167,  0.01848727, ...,  0.00358183,
        -0.07254394,  0.04349626],
       [ 0.01608559, -0.05598367,  0.00448521, ...,  0.09021292,
        -0.03025921,  0.02493638],
       [ 0.03191034, -0.08993487,  0.02811484, ...,  0.09555593,
        -0.06741834,  0.06182566],
       ...,
       [ 0.12486743, -0.02346124, -0.06799407, ..., -0.16489714,
        -0.01889685,  0.2138289 ],
       [-0.03117347, -0.0073445 ,  0.0039943 , ...,  0.0189672 ,
        -0.0324156 , -0.00649395],
       [-0.03760091, -0.00887778,  0.01875546, ...,  0.0008709 ,
        -0.04518886,  0.01807308]], dtype=float32)
Epoch: 0013 train_loss= 1.85771 train_acc= 0.70000 val_loss= 1.90271 val_acc= 0.54600 time= 0.01686
(2708, 7)
array([[-0.05386529, -0.03196117,  0.01578466, ...,  0.05450289,
        -0.04290227,  0.00889561],
       [-0.02069727, -0.07730893,  0.05765935, ...,  0.10965481,
        -0.02175076, -0.01751099],
       [-0.04738721, -0.10743302,  0.03981532, ..., 

Epoch: 0024 train_loss= 1.72883 train_acc= 0.81429 val_loss= 1.84614 val_acc= 0.61600 time= 0.01673
(2708, 7)
array([[-0.04430712, -0.2603249 ,  0.10650712, ..., -0.05074069,
        -0.25741678,  0.22850983],
       [ 0.13800164,  0.09826732, -0.10256086, ...,  0.28716397,
        -0.26564658, -0.18929067],
       [ 0.03547953, -0.06990235, -0.0821183 , ...,  0.24387151,
        -0.19424485, -0.13048777],
       ...,
       [-0.02003883, -0.03623144, -0.11422893, ...,  0.04286476,
         0.05484992,  0.03068238],
       [-0.01375778, -0.16982809,  0.03725303, ...,  0.08907483,
        -0.17330432, -0.00784375],
       [-0.01824391, -0.1356586 ,  0.01572311, ...,  0.06932537,
        -0.12033187, -0.00555988]], dtype=float32)
Epoch: 0025 train_loss= 1.71896 train_acc= 0.77143 val_loss= 1.84003 val_acc= 0.62200 time= 0.01722
(2708, 7)
array([[-0.12895203, -0.22194822,  0.1897312 , ...,  0.01780096,
        -0.20381807, -0.00077494],
       [ 0.00634599, -0.13641489,  0.05001834, ..., 

Epoch: 0036 train_loss= 1.56398 train_acc= 0.85000 val_loss= 1.76040 val_acc= 0.69000 time= 0.01660
(2708, 7)
array([[-0.1053993 , -0.30304712, -0.02298597, ..., -0.01097338,
        -0.23198786,  0.18509556],
       [ 0.07223165, -0.32705152, -0.13210656, ...,  0.58929217,
        -0.4311337 , -0.22646485],
       [ 0.14899021, -0.35030586, -0.1488943 , ...,  0.42413333,
        -0.43127403, -0.34119534],
       ...,
       [ 0.04317997,  0.0121012 , -0.18755105, ...,  0.06517026,
        -0.03949709, -0.06085455],
       [-0.24097617, -0.1941807 ,  0.23114882, ...,  0.26845905,
        -0.46736676, -0.1431068 ],
       [-0.14933904, -0.13182053,  0.0530512 , ...,  0.24943106,
        -0.35616523, -0.07566731]], dtype=float32)
Epoch: 0037 train_loss= 1.54257 train_acc= 0.84286 val_loss= 1.75245 val_acc= 0.69400 time= 0.01871
(2708, 7)
array([[-1.94810957e-01, -3.08060229e-01,  2.98259854e-02, ...,
         1.74861655e-01, -3.34949851e-01, -5.54412603e-04],
       [ 7.65877292e-02, -4.

(2708, 7)
array([[-0.8214725 , -0.41456982,  0.25239345, ...,  0.4140569 ,
        -0.65792465, -0.2550063 ],
       [ 0.03608322, -0.21496211, -0.41667247, ...,  1.4634688 ,
        -0.7489683 , -0.7301749 ],
       [-0.07550604, -0.6679529 , -0.12987919, ...,  1.0073197 ,
        -0.6898366 , -0.55433124],
       ...,
       [ 0.09024477, -0.14407413, -0.39068553, ..., -0.12969428,
        -0.03665304,  0.25414354],
       [-0.25647238, -0.46771872,  0.13015051, ...,  0.16403265,
        -0.45510483, -0.06862271],
       [-0.13254079, -0.36263818,  0.02131477, ...,  0.23291542,
        -0.40201733, -0.08514239]], dtype=float32)
Epoch: 0048 train_loss= 1.37266 train_acc= 0.87857 val_loss= 1.65688 val_acc= 0.73400 time= 0.01783
(2708, 7)
array([[ 0.32455793, -0.93488204, -0.27473313, ...,  0.22253177,
        -0.75358105,  0.39912784],
       [ 0.30590937,  0.125669  , -0.5328171 , ...,  1.2528256 ,
        -0.8840853 , -0.8519206 ],
       [ 0.1540274 , -0.20682684, -0.26605883, ..., 

...

Epoch: 0138 train_loss= 0.71910 train_acc= 0.96429 val_loss= 1.15209 val_acc= 0.78200 time= 0.01788
(2708, 7)
array([[-1.2926381 , -0.44977096, -0.38327634, ...,  0.5966561 ,
        -1.8299234 , -0.89214075],
       [-0.24675052, -1.2809918 , -0.7367503 , ...,  2.332423  ,
        -1.6880728 , -1.1094863 ],
       [-0.4897136 , -0.9413198 , -1.7481805 , ...,  2.400352  ,
        -1.4602939 , -1.1502411 ],
       ...,
       [-2.5991082 ,  1.2246971 , -1.4651613 , ..., -0.7720859 ,
         1.2687682 ,  0.72017646],
       [-0.39299116, -1.4138422 , -0.2820667 , ...,  1.1484311 ,
        -1.8068008 , -0.7852993 ],
       [-0.527384  , -0.8812151 , -0.6103603 , ...,  0.992326  ,
        -1.0370212 , -0.99857676]], dtype=float32)
Epoch: 0139 train_loss= 0.76463 train_acc= 0.94286 val_loss= 1.15014 val_acc= 0.78200 time= 0.01943
(2708, 7)
array([[-2.6752062 , -1.6218498 ,  0.18786061, ...,  0.15396377,
        -3.254794  ,  0.11418067],
       [-0.47274894, -1.0218471 , -0.28353053, ..., 

Epoch: 0149 train_loss= 0.67701 train_acc= 0.96429 val_loss= 1.13657 val_acc= 0.78200 time= 0.01890
(2708, 7)
array([[-2.0881596 ,  0.229903  ,  0.35757518, ...,  0.40115303,
        -1.9818687 , -0.74112886],
       [ 2.37157   , -0.87233746, -2.4148483 , ...,  2.8679821 ,
        -2.2787004 , -0.5809328 ],
       [ 0.66185915, -0.91585934, -1.810303  , ...,  3.1214128 ,
        -2.0553346 , -1.5321139 ],
       ...,
       [ 1.1977539 , -0.04520237, -0.904557  , ..., -0.7961125 ,
         0.3509591 , -0.39024106],
       [-1.3304051 ,  0.46219203, -0.70816886, ...,  0.54393494,
        -0.41913283, -1.0842822 ],
       [-1.7001008 ,  0.6218404 , -0.40478227, ...,  0.34162316,
        -0.60549873, -1.1642908 ]], dtype=float32)
Epoch: 0150 train_loss= 0.68453 train_acc= 0.94286 val_loss= 1.13420 val_acc= 0.78200 time= 0.01782
(2708, 7)
array([[-2.341091  , -0.6828706 , -0.6516477 , ...,  0.33300155,
        -1.3801079 , -0.55530715],
       [ 2.656722  , -0.3577742 , -3.2180338 , ..., 

(2708, 7)
array([[-1.5430899 , -0.6107104 , -0.46108353, ...,  0.67300624,
        -2.3625972 , -0.65391266],
       [ 0.49759302, -1.8883294 , -1.1217015 , ...,  2.2450666 ,
        -0.4005026 , -0.24865156],
       [-0.34515566, -1.7193397 , -0.71333444, ...,  1.9600508 ,
        -1.00199   , -1.0365154 ],
       ...,
       [ 0.01130541,  0.29684204, -0.25185198, ..., -0.8899796 ,
        -0.9480002 ,  0.33366376],
       [-1.2830334 ,  0.07236215,  0.12594947, ...,  0.398876  ,
        -1.2568628 , -0.28441104],
       [-1.3962221 ,  0.01112491, -0.11965457, ...,  0.37799048,
        -1.358918  , -0.3959569 ]], dtype=float32)
Epoch: 0163 train_loss= 0.67199 train_acc= 0.95000 val_loss= 1.10098 val_acc= 0.78200 time= 0.01686
(2708, 7)
array([[-0.94841194, -1.0025436 , -0.3534432 , ..., -1.287605  ,
        -1.54425   ,  0.68665016],
       [ 0.5699207 , -2.2134526 , -0.70991147, ...,  2.8899624 ,
        -2.371469  , -2.1718152 ],
       [-0.68765366, -2.2828698 ,  0.16166651, ..., 

(2708, 7)
array([[-1.4121854 , -1.1514266 , -0.4639025 , ...,  0.2479868 ,
        -1.2229333 , -0.25774533],
       [ 1.3683968 , -1.2500863 , -1.6500314 , ...,  3.8571439 ,
        -2.2844884 , -1.413219  ],
       [ 0.68191195, -1.3683674 , -1.5276102 , ...,  3.7232594 ,
        -2.5763783 , -2.4609132 ],
       ...,
       [ 0.14288782, -0.6934314 , -1.3080862 , ..., -1.5800565 ,
         0.6880878 ,  0.55276394],
       [-0.594095  , -0.67667115, -0.5397018 , ...,  0.58030444,
        -0.8155116 , -1.0579822 ],
       [ 0.11737207, -1.1661686 , -0.81524324, ...,  0.18410544,
        -0.6969788 , -0.5342893 ]], dtype=float32)
Epoch: 0177 train_loss= 0.66996 train_acc= 0.96429 val_loss= 1.08688 val_acc= 0.78400 time= 0.01739
(2708, 7)
array([[-1.7458831 , -1.4282933 ,  0.29182822, ...,  0.33789843,
        -1.8628731 , -1.5744237 ],
       [-1.3422937 , -3.1333451 ,  0.5462423 , ...,  4.322107  ,
        -2.5958483 , -2.7934124 ],
       [-0.769718  , -0.2223175 , -1.1096675 , ..., 

Epoch: 0188 train_loss= 0.59455 train_acc= 0.95000 val_loss= 1.06786 val_acc= 0.78600 time= 0.01689
(2708, 7)
array([[-0.45232707, -0.9511537 , -1.0144546 , ...,  0.66252255,
        -1.8272247 , -0.4722101 ],
       [ 0.8508866 , -1.8877833 , -0.94261086, ...,  3.501129  ,
        -2.8228478 , -2.096024  ],
       [ 0.80725354, -2.230325  , -0.9692705 , ...,  2.8239462 ,
        -3.2866416 , -2.19143   ],
       ...,
       [ 2.332552  ,  0.6006377 , -4.2218666 , ..., -0.7902572 ,
         1.8836789 , -0.23374046],
       [-0.64699084, -0.592185  , -1.1945137 , ...,  1.3647249 ,
        -1.5039124 , -1.2719471 ],
       [-0.477431  , -0.82470506, -0.76267743, ...,  0.6928661 ,
        -2.0300264 , -0.8862606 ]], dtype=float32)
Epoch: 0189 train_loss= 0.62280 train_acc= 0.96429 val_loss= 1.06605 val_acc= 0.78600 time= 0.01750
(2708, 7)
array([[-0.15157709, -1.5685785 , -0.94070166, ..., -0.11566463,
        -0.72046995,  0.41240168],
       [ 1.2162583 , -1.3310022 , -3.1898577 , ..., 